# Let's Build Personalized Website using Agentic Workflows and Inferentia

Notes: 
1. Use the native Jupyter Notebook to leverage the full features of this notebook
2. Ensure you have Bedrock connection with access to Titan Models 

## 1. Import Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import base64
import boto3
import io
import os
import json
import re
import importlib
import shutil
import re
import numpy as np
import matplotlib.pyplot as plt

from src import Bedrock

from ipywidgets import widgets
from IPython.core.display import display, HTML
from PIL import Image


In [ ]:
#!pip install ipywidgets

In [ ]:
importlib.reload(Bedrock)

### Useful functions

In [ ]:
# fetch context from the response
def get_contexts(retrievalResults):
    total_text = ""
    for result in retrievalResults.get("retrievalResults", []):
        content = result.get("content", {})
        text = content.get("text", "")
        # Remove the metadataAttributes content from the text
        text = re.sub(r'\{.*?metadataAttributes.*?\}', '', text)
        total_text += text
    return total_text

In [ ]:
def build_profile(UserProfile):
    profile = f"Your customer is {UserProfile['Name']} located in {UserProfile['Location']}. They are in {UserProfile['Industry']} industry. Their mission statement: `{UserProfile['Mission']}.'"
    return profile

In [ ]:
def get_labels_and_desc(pattern, text):  
    matches = re.findall(pattern, text)
    myList = [] #(label, description)
    for match in matches:
        title = match[0]
        description = match[1]
        myList.append((title,description))
    return myList

In [ ]:
def decode_base64_image(image_string):
  base64_image = base64.b64decode(image_string)
  buffer = io.BytesIO(base64_image)
  return Image.open(buffer)
 
# Display PIL images as grid
def display_image(image=None,width=500,height=500):
    img = image.resize((width, height))
    display(img)

### Declerations

In [ ]:
# create general model inputs 
# do not edit any content in this section

#image_modelId = 'amazon.titan-image-generator-v1'
#model_id = {'Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
#           'Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
#           'StableDiff': 'stability.stable-diffusion-xl-v1'}

In [ ]:
# ENV VARIABLES
# ***** UPDATE FOR THE WORKSHOP *******
kbId = '2TBRS1XZBR'
model_llama3p1_endpoint = 'nous-research-Meta-Llama-3-8B-2024-09-23-19-42-59-162-endpoint'
model_stablediffV1_endpoint = 'huggingface-pytorch-inference-neuronx-2024-10-01-23-05-27-894'

### Checking the endpoint status

In [ ]:
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [ ]:
resp = sm_client.describe_endpoint(EndpointName=model_llama3p1_endpoint)
print(f"Status for {model_llama3p1_endpoint}: {resp['EndpointStatus']}")

In [ ]:
resp = sm_client.describe_endpoint(EndpointName=model_stablediffV1_endpoint)
print(f"Status for {model_stablediffV1_endpoint}: {resp['EndpointStatus']}")

### Example User Profiles

In [ ]:
Profiles = {
    'Construction-Example': {
        'Name': 'Example Corp Construction Inc',
        'Industry': 'Construction',
        'Location': 'New York City, NY',
        'Mission': 'Building a sustainable future for New York'
    },
    'Manufacturing-Example': {
        'Name': 'Example Corp Manuf LLC',
        'Industry': 'Manufacturing',
        'Location': 'San Jose, CA',
        'Mission': 'Building the next generation Electric Vehicles'
    },
    'Mining-Example': {
        'Name': 'Example Corp Mining Inc',
        'Industry': 'Mining',
        'Location': 'Bisbee, AZ',
        'Mission': 'Extracting the value for America'
    }
}

### INPUT

Offerings

In [ ]:
with open("./references/offerings.json", "r") as file:
    offerings = json.load(file)

Testimonials

In [ ]:
with open('./references/Testimonials.json', 'r') as file:
    testimonials = json.load(file)

Now a user signs in (ITERATION)

In [ ]:
dropdown = widgets.Dropdown(
       options=['Manufacturing', 'Construction', 'Mining'],
       value='Manufacturing',
       description='Select your example industry:',
   )

In [ ]:
display(dropdown)

In [ ]:
customer= Profiles[dropdown.value+'-Example']
offering = re.sub(r'\n#?', ' ', offerings[customer['Industry']])
project = dropdown.value + "_Website_Test"

In [ ]:
print(f"Selected Customer:\n{customer}")

### Runtime

#### 1) Natural Language description of customer profile

In [ ]:
customer['Description'] = build_profile(customer)
print(customer['Description'])

#### 2) Retreieve painpoints of customer industry / any context
This flow uses RAG to retreive documents

In [ ]:
query = f"{customer['Description']}\nList the industry pain-points and challenges"
print(f"Here is your customer:\n{query}")

In [ ]:
# Let's find the context for the painpoints and challenges for this customer using RAG.
context_painpoints = Bedrock.dec_retrieve(query, kbId, numberOfResults=5)
contexts_painpoints = get_contexts(context_painpoints)
contexts_painpoints = re.sub(r'\n#?', ' ', contexts_painpoints)
print(f"These are the applicable pain-points/challenges:\n{contexts_painpoints}")

### 3) Prompting AI to describe a website personalized for this profile and background
Here the prompting is critical. Please see the guidance provided to the LLM. We also ask the response to be in two parts; 1/ Description, 2/ Visual Elements. The second part is needed for an easier traffic move to image generation or finding image.

#### 3.1) Finding the possible painpoints using LLM

In [ ]:
prompt = f"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are an experienced business consultant.
<|eot_id|><|start_header_id|>user<|end_header_id|>
*** Research Document ***
{contexts_painpoints}
*** End of Research Document ***


Question: {customer['Description']} Based on the above research, list top-3 challenges and explain we understand their challanges. Do not use any company name.
Provide your response in bullet points without any preamble (e.g. • a pain-point: one sentence description, • pain-point: one sentence description, etc.). 

"""
params = {"max_new_tokens": 512, "do_sample": False}

In [ ]:
%%time
response_model= smr_client.invoke_endpoint(
    EndpointName=model_llama3p1_endpoint,
    Body=json.dumps({"inputs": prompt, "parameters": params}),
    ContentType="application/json",
)
response_painpoints = json.load(response_model['Body'])
Top_pain_points_text= response_painpoints['generated_text'].split('<|eot_id|>')[0].replace('\n',', ')
print(f"Raw Output - top painpoints:\n{Top_pain_points_text}\n")

In [ ]:
# Convert this into a function
pattern = r'•\s*(.*?):\s*(.*?)\.'
Top_painpoints = get_labels_and_desc(pattern, Top_pain_points_text)
[print(f"{key}: {value}\n")for (key, value) in Top_painpoints]

#### 3.2) Finding the personalized offerings using LLM

In [ ]:
prompt = f"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are an experienced business consultant.
<|eot_id|><|start_header_id|>user<|end_header_id|>
*** Offerings Document ***
{offering}
*** Offerings Document ***


Question: {customer['Description']} Their top-3 painpoints are {Top_painpoints}.

List and briefly advertise the top-3 offerings for {customer['Name']} based on the offerings document above.
Provide your list in bullet points without any preamble (e.g. • an offering: short description, • an offering: short description, ...). 
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
params = {"max_new_tokens": 384, "do_sample": False}

In [ ]:
%%time
response_model = smr_client.invoke_endpoint(
    EndpointName=model_llama3p1_endpoint,
    Body=json.dumps({"inputs": prompt, "parameters": params}),
    ContentType="application/json",
)
response_offerings = json.load(response_model['Body'])
Top_offerings_text= response_offerings['generated_text'].split('<|eot_id|>')[0].replace('\n',', ')
print(f"Raw Output - top offerings:\n{Top_offerings_text}\n")

In [ ]:
# Convert this into a function
pattern = r'•\s*(.*?):\s*(.*?)\.'
Top_offerings = get_labels_and_desc(pattern, Top_offerings_text)
[print(f"{key}: {value}\n")for (key, value) in Top_offerings]

#### 3.3) Art Design

In [ ]:
prompt = f"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are a UI/UX art designer personalizing visual website content such as hero image and icons tailored to customer profile, personalized offerings and pain-points. 
Your task is to describe a hero image and six icons. For the Hero Image, design an attention-grabbing image that relates to the customer's industry, location, and mission. For icons, design simple, memorable icons that visually represent the concepts.

Customer Profile:
- Industry: {customer['Industry']}
- Location: {customer['Location']}
- Company Motto/Mission: {customer['Mission']}

For the Hero Image:
- Describe a scene that represents the customer's industry and mission
- Include elements that reflect the customer location
- Mention 1-2 key visual elements

For each icon:
- Describe a simple, recognizable shape or symbol
- Concisely describe the key visual element
- Call our that this is an 'icon'

Each icon should be distinct yet part of a cohesive set in style and color.

<|eot_id|><|start_header_id|>user<|end_header_id|>

Provide your response in the following structured format:

** Hero Image[Description in 1-2 sentences]**
** {Top_painpoints[0][0]} Icon [Description in one sentence]**
** {Top_painpoints[1][0]} Icon [Description in one sentence]**
** {Top_painpoints[2][0]} Icon [Description in one sentence]**
** {Top_offerings[0][0]} Icon [Description in one sentence]**
** {Top_offerings[1][0]} Icon [Description in one sentence]**
** {Top_offerings[2][0]} Icon [Description in one sentence]**

Provide your response using the exact format above, without any preamble.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
params = {"max_new_tokens": 512, "do_sample": False}

In [ ]:
%%time
response_model = smr_client.invoke_endpoint(
    EndpointName=model_llama3p1_endpoint,
    Body=json.dumps({"inputs": prompt, "parameters": params}),
    ContentType="application/json",
)
response_art= json.load(response_model['Body'])
art_descriptions_text = response_art['generated_text'].split('<|eot_id|>')[0]\
                        .replace('\n',', ').replace(', ,', '').replace('**,','**')
print(f"Raw Output - top offerings:\n{art_descriptions_text}\n")

In [ ]:
# Creating the variable for the art descriptions that will help us to create a better prompt later
art_descriptions = art_descriptions_text.split("**")[1:]
art_descriptions = {art_descriptions[i].strip():{"Description":art_descriptions[i+1].strip(), \
                    'FileName':art_descriptions[i].replace(' ','')+'.jpg'} for i in range(0,int(len(art_descriptions)),2)}

In [ ]:
print(f"Here is the art and their descriptions:\n {json.dumps(art_descriptions, indent=4)}")

#### Call to action, hero labels and description - final touches

In [ ]:
prompt = f"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are creating engaging website content for AnyCompany Consulting, which is a business consulting company. 
Your task is to generate four specific pieces of text based on the customer profile and offerings provided.

Customer Profile:
- Industry: {customer['Industry']}
- Location: {customer['Location']}
- Company Motto/Mission: {customer['Mission']}

Hero image: {art_descriptions['Hero Image']['Description']}

Industry painpoints:
- {Top_painpoints[0][0]}
- {Top_painpoints[1][0]}
- {Top_painpoints[2][0]}

AnyCompany Consulting offerings:
- {Top_offerings[0][0]}
- {Top_offerings[1][0]}
- {Top_offerings[2][0]}

<|eot_id|><|start_header_id|>user<|end_header_id|>

Generate the following four pieces of text:
1. Hero Label: A short, attention-grabbing phrase to work with AnyCompany Consulting (5-7 words)
2. Hero Text: A brief explanation expanding on the label (15-20 words)
3. Call-to-Action Button: A short, action-oriented phrase (3-5 words)
4. Call-to-Action Explanation: A motivating statement to encourage engagement with AnyCompany Consulting (15-20 words)

Use this exact format for your response:

** Hero Label **
** Hero Text **
** Call-to-Action Button **
** Call-to-Action Explanation **

Ensure each piece of text relates to the customer's industry, pain points, and the consulting offerings. 
Focus on engaging the customer and encouraging them to work with AnyCompany Consulting.
Provide your response using the exact format above, without any preamble.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
params = {"max_new_tokens": 512, "do_sample": False}

In [ ]:
%%time
response_model = smr_client.invoke_endpoint(
    EndpointName=model_llama3p1_endpoint,
    Body=json.dumps({"inputs": prompt, "parameters": params}),
    ContentType="application/json",
)
response_text= json.load(response_model['Body'])
text_descriptions_text = response_text['generated_text'].split('<|eot_id|>')[0]\
                        .replace('\n',', ').replace(', ,', '').replace('**, ','**')
print(f"Raw Output - Description of critical text:\n{text_descriptions_text}\n")

In [ ]:
text_descriptions_lst = text_descriptions_text.split('**')
text_descriptions_lst = [x for x in text_descriptions_lst if x is not '']
text_descriptions = {"HeroLabel":text_descriptions_lst[0], "HeroText":text_descriptions_lst[1], "CTALabel":text_descriptions_lst[2] ,"CTAText":text_descriptions_lst[3]}

In [ ]:
print(text_descriptions)

#### 5) Image Creation based on Image Descriptions

##### 5.1) Creating the imagery from the descriptions
Notice that if the image is "icon" then 512 x 512 otherwise 1024 x 1024

In [ ]:
%%time
# WE HAVE TO OPTIMIZE THIS FOR ICONS.
from sagemaker.huggingface.model import HuggingFacePredictor
predictor_SDv1 = HuggingFacePredictor(model_stablediffV1_endpoint)

images = []
image_files = []
for art, content in art_descriptions.items():
    if 'Icon' in art:
        art_height = 512
        art_width = 512
        inf_steps = 20
    else:
        art_height = 1024
        art_width = 1024
        inf_steps = 30
    
    params = {
    "num_inference_steps" : inf_steps,
    "negative_prompt" : "disfigured, ugly, deformed, unprofessional"
    }
    
    prompt = content['Description']
    response = predictor_SDv1.predict(data={"inputs": prompt,"parameters":params})

    image = decode_base64_image(response["generated_images"][0])
    image_files.append(content['FileName'])

    # Save the image
    image.save(f"{content['FileName']}")
    images.append(np.array(image))

In [ ]:
print("Number of visual assets created: {}".format(len(image_files)))

##### b) Invoke Stable Diffusion
Here we assumed that the image description is the prompt for Text-to-Image. We successively call the APIs and then save the files.

In [ ]:
shutil.copy("./references/anycompany_logo.jpg", "./anycompany_logo.jpg")

Let's plot the images generated

In [ ]:
# Plot the images
fig, axs = plt.subplots(1, len(images), figsize=(30, 20))

for i, (image, ax, filename) in enumerate(zip(images, axs, image_files)):
    ax.imshow(image)
    ax.set_title(filename, fontsize=16)
    ax.axis('off')

plt.tight_layout()
plt.show()

#### 6) Let's Generate our HTML file

First, we need to extract the response for the Section 1: Detailed Description from *response_personalized_website*

In [ ]:
CSS_section = open('./references/CSS.txt', 'r').read()
CSS_section = CSS_section.replace("hero-image.jpg", art_descriptions['Hero Image']['FileName'])
JS_section = open('./references/JS.txt', 'r').read()
full_html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>AnyCompany Consulting Personalized Page</title>
  <style>
{CSS_section}
  </style>
</head>
<body>
  <header>
    <div class="logo">
      <img src="anycompany_logo.jpg" alt="AnyCompany Consulting">
    </div>
    <nav>
      <ul>
        <li><a href="#">Home</a></li>
        <li><a href="#">About Us</a></li>
        <li><a href="#">Services</a></li>
        <li><a href="#">Projects</a></li>
        <li><a href="#">Resources</a></li>
        <li><a href="#">Contact</a></li>
      </ul>
    </nav>
  </header>
  <main>
    <section class="hero">
      <h1>{text_descriptions['HeroLabel']}</h1>
      <p>{text_descriptions['HeroText']}</p>
    </section>
    <section class="offerings">
      <h2>Our Tailored Solutions for Your {customer['Industry']} Needs</h2>
      <div class="offerings-grid">
        <div class="offering-card">
          <img src="{art_descriptions[Top_offerings[0][0]+' Icon']['FileName']}" alt="{Top_offerings[0][0]}">
          <h3>{Top_offerings[0][0]}</h3>
          <p>{Top_offerings[0][1]}</p>
        </div>
        <div class="offering-card">
          <img src="{art_descriptions[Top_offerings[1][0]+' Icon']['FileName']}" alt="{Top_offerings[1][0]}">
          <h3>{Top_offerings[1][0]}</h3>
          <p>{Top_offerings[1][1]}</p>
        </div>
        <div class="offering-card">
          <img src="{art_descriptions[Top_offerings[2][0]+' Icon']['FileName']}" alt="{Top_offerings[2][0]}">
          <h3>{Top_offerings[2][0]}</h3>
          <p>{Top_offerings[2][1]}</p>
        </div>
      </div>
    </section>
    <section class="pain-points">
      <h2>Addressing Your {customer['Industry']} Challenges</h2>
      <div class="pain-points-grid">
        <div class="pain-point-card">
          <img src="{art_descriptions[Top_painpoints[0][0]+' Icon']['FileName']}" alt="{Top_painpoints[0][0]}">
          <h3>{Top_painpoints[0][0]}</h3>
          <p>{Top_painpoints[0][1]}</p>
        </div>
        <div class="pain-point-card">
          <img src="{art_descriptions[Top_painpoints[1][0]+' Icon']['FileName']}" alt="{Top_painpoints[1][0]}">
          <h3>{Top_painpoints[1][0]}</h3>
          <p>{Top_painpoints[1][1]}</p>
        </div>
        <div class="pain-point-card">
          <img src="{art_descriptions[Top_painpoints[2][0]+' Icon']['FileName']}" alt="{Top_painpoints[2][0]}">
          <h3>{Top_painpoints[2][0]}</h3>
          <p>{Top_painpoints[2][1]}</p>
        </div>
      </div>
    </section>
    <section class="testimonials">
      <h2>What Our Clients Say</h2>
      <div class="testimonial-container">
        <div class="testimonial" style="display: none;">
          <p class="testimonial-text">{testimonials['testimonials'][0]['text']}</p>
          <p class="testimonial-author">- {testimonials['testimonials'][0]['author']}, {testimonials['testimonials'][0]['position']} from {testimonials['testimonials'][0]['company']}</p>
        </div>
        <div class="testimonial" style="display: none;">
          <p class="testimonial-text">{testimonials['testimonials'][1]['text']}</p>
          <p class="testimonial-author">- {testimonials['testimonials'][1]['author']}, {testimonials['testimonials'][1]['position']} from {testimonials['testimonials'][1]['company']}</p>
        </div>
        <div class="testimonial" style="display: none;">
          <p class="testimonial-text">{testimonials['testimonials'][2]['text']}</p>
          <p class="testimonial-author">- {testimonials['testimonials'][2]['author']}, {testimonials['testimonials'][2]['position']} from {testimonials['testimonials'][2]['company']}</p>
        </div>
      </div>
    </section>
    <section class="cta">
      <h2>{text_descriptions['CTALabel']}</h2>
      <p>{text_descriptions['CTAText']}</p>
      <a href="#" class="cta-button">Get Started</a>
    </section>
  </main>
  <footer>
    <p>&copy; 2023 AnyCompany Consulting. All rights reserved.</p>
    <p>
      <a href="#">Address</a> | <a href="#">Phone</a> | <a href="#">Email</a>
    </p>
  </footer>
  <script>
  {JS_section}
  </script>
</body>
</html>
"""

Here we will use Haiku to create the HTML file.

#### 7) (OPTIONAL) - Testing our website with CodeLama

In [ ]:
# prompt = f"""
# <|begin_of_text|>
# <|start_header_id|>system<|end_header_id|>
# You are a quality-assurance front-end developer. 
# You look into "main.hmtl" file including HTML, CSS and JavaScript and see if there are issues listed in the below testing requirements. If you see issues, you correct the file. You provide the final "main.html".env

# You check and correct the following:
# 1. Image size and format:
#    - Check if icon images in the Offerings and Pain-points sections are 64x64 pixels.
#    - Verify that image file sizes are optimized for web use.

# 2. Accessibility:
#    - Check if all images have appropriate alt text.
#    - Ensure proper heading hierarchy (h1, h2, h3, etc.).
#    - Verify sufficient color contrast between text and background.

# 3. Responsive design:
#    - Check if the layout adjusts properly for different screen sizes.
#    - Ensure images are responsive and scale appropriately.

# 4. Code structure and formatting:
#    - Verify proper HTML5 structure and use of semantic elements.
#    - Check for consistent indentation and formatting.

# 5. CSS optimization:
#    - Look for redundant or overspecific CSS rules.
#    - Suggest using CSS variables for repeated values (e.g., colors, fonts).

# 6. Performance:
#    - Check for render-blocking resources in the <head> section.
#    - Suggest asynchronous loading of non-critical CSS and JavaScript.

# 7. SEO:
#    - Verify presence of meta description tag.
#    - Check if page title is descriptive and appropriate length.

# 8. Content:
#    - Check for spelling and grammar errors.
#    - Ensure consistent use of terminology and brand voice.

# 9. Functionality:
#    - Verify all links are working and point to correct destinations.
#    - Check if interactive elements (e.g., navigation, buttons) function properly.

# 10. Best practices:
#     - Suggest using minified CSS and JavaScript for production.
#     - Recommend implementing proper caching headers for static assets.

# <|eot_id|><|start_header_id|>user<|end_header_id|>

# Go over the below "main.html" and correct and provide the corrected "main.html". 
# *** 'main.html' ***
# {full_html}
# *** end of 'main.html' ***

# Provide the file without any preamble.
# <|eot_id|><|start_header_id|>assistant<|end_header_id|>
# """
# params = {"max_new_tokens": 8000, "do_sample": False}

# response_model = smr_client.invoke_endpoint(
#     EndpointName=model_llama3p1_endpoint,
#     Body=json.dumps({"inputs": prompt, "parameters": params}),
#     ContentType="application/json",
# )
# response_text= json.load(response_model['Body'])
# text_descriptions_text = response_text['generated_text'].split('<|eot_id|>')[0]\
#                         .replace('\n',', ').replace(', ,', '').replace('**, ','**')
# print(f"Raw Output - Description of critical text:\n{text_descriptions_text}\n")

#### Putting the assets into a single folder

In [ ]:
#Creating the folder for our project
try:
    os.mkdir(project)
    print(f"Project Folder: {project}" + '_reInvent')
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
from IPython.display import HTML

In [ ]:
with open('main.html', 'w', encoding='utf-8') as file:
    file.write(full_html)

In [ ]:
project_files = image_files.copy()
project_files.append('main.html')
project_files.append('anycompany_logo.jpg')

In [ ]:
for file in project_files:
    destination = project + "/" + file
    shutil.move(file, destination)